In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from shapely.geometry import Point, LineString, Polygon, mapping
import numpy as np
import folium
from folium.features import DivIcon
from folium.plugins import HeatMap

In [41]:
def get_hexagon(city):
    #city = ['DIDI','NYU'][1]
    if city=='NYU':
        r=6371
        per_lon=r/360*2*3.14159*np.cos(np.pi*(40.5/180))
        per_lat=r/360*2*3.14159
        per_lonlat=np.array([per_lon,per_lat])
        corner=[
        [-74.04,40.91553],
        [-74.04,40.58 ],
        [-73.72 ,40.91553],
        [-73.72 ,40.58]
        ]
    elif city=='DIDI':
        r=6371
        per_lon=r/360*2*3.14159*np.cos(np.pi/6)
        per_lat=r/360*2*3.14159
        per_lonlat=np.array([per_lon,per_lat])
        corner=[
        [103.9521,30.7900],
        [103.9521,30.5400],
        [104.1934,30.7900],
        [104.1934,30.5400]
        ]
    def cal_dis(array1,array2):
        return np.sqrt(np.sum((np.abs(array1-array2)*per_lonlat)**2))
    # 绘制六边形的中心点
    corner=np.array(corner)
    radius=cal_dis(corner[0],corner[1])/13
    lat_num=int(cal_dis(corner[0],corner[1])/radius)
    lon_num=int(round(cal_dis(corner[0],corner[2])/(radius*np.sqrt(3)/2)))
    lat_per_radius=(corner[0,1]-corner[1,1])/lat_num
    lon_per_radius=(corner[2,0]-corner[0,0])/lon_num
    #print(radius, lat_num, lon_num, lat_per_radius, lon_per_radius)
    #assert lat_num==10 and lon_num==13 , print(lat_num,lon_num)
    grid_hexagon=np.zeros((lat_num,lon_num,2))
    center = np.array(corner[0])
    for i in range(lat_num):
        for j in range(lon_num):
            grid_hexagon[i,j][0]=lon_per_radius*j+center[0]
            if j%2==0:
                grid_hexagon[i,j][1]=-lat_per_radius*(i+0.5)+center[1]
            else:
                grid_hexagon[i,j][1]=-lat_per_radius*i+center[1]

    grid_hexagon = grid_hexagon.reshape(-1,2)

    lon_per_radius,lat_per_radius = lon_per_radius*0.9, lat_per_radius*0.9

    def get_hexagon_poly(center):
        hexa = []   # [lon,lat]
        hexa.append([center[0]-lon_per_radius/3, center[1]+lat_per_radius/2])
        hexa.append([center[0]+lon_per_radius/3, center[1]+lat_per_radius/2])
        hexa.append([center[0]+lon_per_radius/3*2, center[1]])
        hexa.append([center[0]+lon_per_radius/3, center[1]-lat_per_radius/2])
        hexa.append([center[0]-lon_per_radius/3, center[1]-lat_per_radius/2])
        hexa.append([center[0]-lon_per_radius/3*2, center[1]])
        return hexa

    poly_hexa = np.array([get_hexagon_poly(g) for g in grid_hexagon])

    def rorate(data, theta):
        x,y = data[...,0], data[...,1]
        newx = np.cos(theta)*x-np.sin(theta)*y
        newy = np.cos(theta)*y+np.sin(theta)*x
        return np.stack([newx,newy],axis=-1)

    # 旋转网格
    #poly_hexa = rorate(poly_hexa-np.mean(grid_hexagon,axis=0,keepdims=True)[:,None,:],+np.pi/30*16)+np.mean(grid_hexagon,axis=0,keepdims=True)[:,None,:]
    #grid_hexagon = rorate(grid_hexagon-np.mean(grid_hexagon,axis=0,keepdims=True),+np.pi/30*16)+np.mean(grid_hexagon,axis=0,keepdims=True)
    #poly_hexa = [Polygon(get_hexagon_poly(g)) for g in grid_hexagon]

    grid_hexagon = grid_hexagon[...,[1,0]]
    poly_hexa = poly_hexa[...,[1,0]]

    return grid_hexagon,poly_hexa

def get_order(city):
    if city=='NYU':
        data_dir = '../data/NYU/NYU_grid143.pkl'
    else:
        data_dir =  '../data/DiDi/DiDi_day1_grid121.pkl'
    with open(data_dir, 'rb') as handle:
        data_param = pickle.load(handle) 
    neighbor = data_param['neighbor']    # neighbor>=100 表示不可达的订单
    order_param = data_param['order']   # shape=(11,11,144)  表示 (出发地，目的地，出发时间)
    M,N = data_param['shape']
    np.random.seed(0)
    commute1 = order_param.astype(np.float32)
    commute1[(neighbor==0)] = 0
    index = commute1>=3
    commute1[index] = (commute1[index]-3)*0.2+3
    index = commute1>=2
    commute1 = np.round(commute1/2+0.1)
    commute1[neighbor==100]=0
    random_delete = np.random.randint(0,3,(commute1.shape))
    random_delete[commute1.sum(-1).sum(-1)<3000] = 0
    commute1 = commute1-random_delete
    commute1[commute1<0] = 0
    random_delete = np.random.randint(0,2,(commute1.shape[1],commute1.shape[2]))
    commute1[68]-=random_delete
    commute1[commute1<0] = 0
    # 添加与邻居相关的随机数据
    random_grid_num = np.random.randint(1,6,M*N)
    random_prob = np.zeros((M*N,M*N))
    prob_list = [0.05,0.2,0.4,0.15,0.1,0.05,0.05]
    for k in range(7):
        index= neighbor==k
        random_prob+= prob_list[k]/np.sum(index,axis=-1,keepdims=True)*index
    random_add = np.zeros(commute1.shape)
    for i in range(M*N):
        sample = np.random.choice(M*N,size = (random_grid_num[i],commute1.shape[-1]),replace=True, p=random_prob[i])    
        for t in range(commute1.shape[-1]):
            random_add[i,sample[:,t],t] = 1
    commute1+=random_add
    # 删除数量多的
    random_delete = np.random.randint(0,2,(commute1.shape))
    #random_delete[random_delete<=1] = 0
    index = commute1.sum(1)<=25
    random_delete = random_delete.swapaxes(1,2)
    random_delete[index] = 0
    random_delete = random_delete.swapaxes(1,2)
    commute1 = commute1-random_delete
    commute1[commute1<0] = 0
    # 全部删除一点点
    random_delete = np.random.randint(0,5,(commute1.shape))
    random_delete[random_delete<=3] = 0
    random_delete[random_delete>3] = 1
    commute1 = commute1-random_delete
    commute1[commute1<0] = 0
    order_param = commute1.astype(np.int32)
    return order_param.sum(-1).sum(-1)

In [43]:
city = ['DIDI','NYU'][1]
# 绘制六边形网格和热力图
grid_hexagon,poly_hexa = get_hexagon(city)  # 获取六边形网格
order_data = get_order(city)                # 获取订单分布
grid_order_data = np.concatenate([grid_hexagon,order_data[:,None]],axis=-1)   # 热力图的数据格式，[纬度,经度,订单数量]

draw_map=folium.Map(
    location=grid_hexagon.mean(0),
    zoom_start=20,
    control_scale=True,
    #tiles='https://webrd02.is.autonavi.com/appmaptile?lang=en&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
    tiles='https://mt.google.com/vt/lyrs=r&x={x}&y={y}&z={z}',      # '谷歌地图
    #tiles = 'https://{s}.tile.openstreetmap.de/tiles/osmde/{z}/{x}/{y}.png',
    #attr='高德-纯英文对照',)
    attr = 'default')
'''
folium.TileLayer(tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
                         attr="&copy; <a href=http://ditu.amap.com/>高德地图</a>",
                         min_zoom=0,
                         max_zoom=19,
                         control=True,
                         show=True,
                         overlay=False,
                         name='baseLayer',
                         ).add_to(draw_map)

folium.TileLayer(tiles='http://rt{s}.map.gtimg.com/realtimerender?z={z}&x={x}&y={y}&type=vector&style=0',
                         attr="&copy; <a href=http://map.qq.com/>腾讯地图</a>",
                         min_zoom=0,
                         max_zoom=19,
                         control=True,
                         show=True,
                         overlay=False,
                         name='baseLayer',
                         ).add_to(draw_map)
'''
folium.LatLngPopup().add_to(draw_map)
# 绘制六边形
for i in range(grid_hexagon.shape[0]):
    folium.Polygon(
        locations = poly_hexa[i] , 
        color='green', 
        weight=1, 
        fill_color='white', 
        fillOpacity=0,
        popup=folium.Popup('grid:{}, OrderNum:{}'.format(i,order_data[i]))
        ).add_to(draw_map)
# 绘制热力图
HeatMap(grid_order_data).add_to(draw_map)
draw_map.save('热力图.html')
draw_map

In [78]:
logdir = '../logs/synthetic/grid143/EnvStat326_OD143_FMRLmerge_Batch2000_Gamma0.97_Lambda0.95_Iter1_Ir0.001_Step144_Ent0.005_Minibatch5_Parallel5mix_MDP0_StateEmb2_Meta04_diffu1_DGCNC_relufeaNor3/distribution.pkl'
#logdir = '../logs/synthetic/grid143/greedy_EnvStat326_20230124_16-08/distribution.pkl'
#logdir = '../logs/synthetic/grid143/random_EnvStat326/distribution.pkl'
#logdir = '../logs/synthetic/grid143/MDP/OD+randomFM_EnvStat326_20230124_16-32/distribution.pkl'
savedir = ''.join([s+'/' for s in logdir.split('/')[:-1]])

with open(logdir,'rb') as f:
    data = pickle.load(f)
reward = data['reward']*10
driver = data['driver']
order = data['order']
gap = np.abs(driver-order)[42:].sum(0)
heat_data = gap
heat_data[heat_data>3000] = 3000

city = ['DIDI','NYU'][1]
# 绘制六边形网格和热力图
grid_hexagon,poly_hexa = get_hexagon(city)  # 获取六边形网格
#order_data = get_order(city)                # 获取订单分布
grid_order_data = np.concatenate([grid_hexagon,heat_data[:,None]],axis=-1)   # 热力图的数据格式，[纬度,经度,订单数量]
base = grid_order_data.mean(0)-0.3
base[2] = 3000
grid_order_data = np.concatenate([grid_order_data,base[None,:]],axis=0)
#grid_order_data[:,2]/=100

color_grad = {
    1:'red',
    0.65:'lime',
    0.2:'blue'
}

draw_map=folium.Map(
    location=grid_hexagon.mean(0),
    zoom_start=20,
    control_scale=True,
    #tiles='https://webrd02.is.autonavi.com/appmaptile?lang=en&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
    tiles='https://mt.google.com/vt/lyrs=r&x={x}&y={y}&z={z}',      # '谷歌地图
    #tiles = 'https://{s}.tile.openstreetmap.de/tiles/osmde/{z}/{x}/{y}.png',
    #attr='高德-纯英文对照',)
    attr = 'default')

folium.LatLngPopup().add_to(draw_map)
# 绘制六边形
for i in range(grid_hexagon.shape[0]):
    folium.Polygon(
        locations = poly_hexa[i] , 
        color='green', 
        weight=1, 
        fill_color='green', 
        fillOpacity=0.1,
        popup=folium.Popup('grid:{}, OrderNum:{}'.format(i,heat_data[i]))
        ).add_to(draw_map)
# 绘制热力图
HeatMap(grid_order_data).add_to(draw_map)
draw_map.save(savedir+'供需热力图.html')
draw_map

In [70]:
logdir = '../logs/synthetic/grid143/MDP/OD+randomFM_EnvStat326_20230124_16-32/MDP.pkl'
savedir = ''.join([s+'/' for s in logdir.split('/')[:-1]])

with open(logdir,'rb') as f:
    data = pickle.load(f)

In [71]:
data

{'value': array([[0.17200232, 0.3898835 , 0.513035  , ..., 0.55225938, 0.08309342,
         0.11388789],
        [0.1932476 , 0.53638715, 0.45193729, ..., 0.39169428, 0.1372212 ,
         0.20471347],
        [0.24766992, 0.51076515, 0.52893885, ..., 0.44347175, 0.23227193,
         0.23535562],
        ...,
        [0.22421027, 0.55492947, 0.80029727, ..., 0.92597274, 0.78300984,
         0.34853499],
        [0.05295329, 0.23766339, 0.57987786, ..., 0.5231692 , 0.25467767,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 'num': array([[231., 231., 231., ..., 231., 209., 209.],
        [ 99., 121., 341., ..., 185., 241., 114.],
        [200., 198., 163., ..., 134., 216., 103.],
        ...,
        [ 44.,  77., 109., ...,  89., 100.,  44.],
        [ 55.,  55., 121., ..., 121.,  98.,  39.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]])}